# Imports

In [1]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import random
import pandas as pd
from google.colab import userdata
import math
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
import json

In [4]:
api_key=userdata.get("HF_API")

In [5]:
# Read the JSON file with multiple objects (line-delimited JSON)
rgb_response_data_without_doc = pd.read_json('/content/en_fact.json', lines=True)
rgb_response_data_doc = pd.read_json('/content/en_fact.json', lines=True)

#Execution for LLM without Documents

In [6]:
def processdata(instance):
    docs = []
    positive = instance['positive_wrong']
    # negative = instance['negative']

    docs = positive  # + negative

    random.shuffle(docs)

    return docs

In [7]:
sys_content = "You are an accurate and reliable AI assistant that can answer questions.Aswer the question "


In [8]:
def response_prompt_wihtout_doc():
    retrieval_prompt= sys_content +" the Question is :\n{QUERY}"
    retrieval_prompt_template = PromptTemplate.from_template(retrieval_prompt)
    return retrieval_prompt_template

In [9]:
def get_hugging_face_llm_response_withoutDocs(question,eval_model,temp):

     # Set up the LLM endpoint
    llm_retrieval = HuggingFaceEndpoint(
        repo_id=eval_model,
        temperature=temp,
        huggingfacehub_api_token=api_key,
    )

    # Combine the retrieval template and the LLM
    llm_retrieval_chain = response_prompt_wihtout_doc() | llm_retrieval

    # Prepare input for the chain
    input_retrieval = {

        "QUERY": question
    }

    # Invoke the chain to generate a response
    try:
        llm_response = llm_retrieval_chain.invoke(input_retrieval)
        return llm_response
    except Exception as e:
        print(f"Error invoking LLM retrieval chain: {e}")
        return None


In [10]:
def checkanswer(prediction, ground_truth):
    prediction = prediction.lower()
    if type(ground_truth) is not list:
        ground_truth = [ground_truth]
    labels = []
    for instance in ground_truth:
        flag = True
        if type(instance)  == list:
            flag = False
            instance = [i.lower() for i in instance]
            for i in instance:
                if i in prediction:
                    flag = True
                    break
        else:
            instance = instance.lower()
            if instance not in prediction:
                flag = False
        labels.append(int(flag))
    return labels

In [13]:
from huggingface_hub import InferenceClient

def get_hugging_face_llm_response_new(documents, question, eval_model, temp, api_key):
    # Set up the Inference Client
    client = InferenceClient(model=eval_model, token=api_key)

    # Prepare input prompt
    input_text = f"{sys_content} Context: {documents}\n\nQuestion: {question}\n\nAnswer:"

    try:
        # Generate response using `text_generation` method
        response = client.text_generation(input_text, temperature=temp, max_new_tokens=1024)
        return response
    except Exception as e:
        print(f"Error invoking LLM retrieval chain: {e}")
        return None

In [11]:
def get_compare_val_answer(llm_answer,ground_truth):
    if 'insufficient information' in llm_answer:
        labels = [-1]

    else:
        labels = checkanswer(llm_answer, ground_truth)

    factlabel = 0

    if 'factual errors' in llm_answer:
        factlabel = 1

    return labels,llm_answer,factlabel

In [12]:
rgb_response_data_without_doc['answer_without_doc'] = None
rgb_response_data_without_doc['label_without_doc'] = None

In [14]:
def create_json(file_name, count, llm_response_for_query, question, labels,ground_truth, fact_level):
    newinstance = {
        "count": count,
        "question": question,
        "answer": llm_response_for_query,
        "groud_truth": ground_truth,
        "label": labels,
        "fact_level": fact_level
    }

    # Append to file
    with open(file_name, 'a', encoding='utf-8') as f:
        f.write(json.dumps(newinstance, ensure_ascii=False) + '\n')
 # Return the JSON object if needed


In [15]:
from huggingface_hub import InferenceClient

def get_hugging_face_llm_response_new_without_doc(question, eval_model, temp, api_key):
    # Set up the Inference Client
    client = InferenceClient(model=eval_model, token=api_key)

    # Prepare input prompt
    input_text = f"{sys_content} Question: {question}"

    try:
        # Generate response using `text_generation` method
        response = client.text_generation(input_text, temperature=temp, max_new_tokens=1024)
        return response
    except Exception as e:
        print(f"Error invoking LLM retrieval chain: {e}")
        return None

In [16]:
execution = 0
insufficient_info = 0 # Initialize the  counter
fact_error_exe=0

results = []  # List to store results for calculating the ratio later
response_model="google/gemma-2-27b-it"  #llama-3.3-70b-versatile,mixtral-8x7b-32768,Qwen/QwQ-32B-Preview,mistralai/Mistral-7B-Instruct-v0.3
temp = 0.5
for index, row in rgb_response_data_without_doc.iterrows():
    # Generate samples based on positive, negative data
    print(index)

    # docs = processdata(row)
    #documents, question, eval_model, temp, api_key
    llm_answer = get_hugging_face_llm_response_new_without_doc( row["query"], response_model, temp,api_key)

    if llm_answer is not None:
        # Compare the LLM answer with the ground truth
        labels, llm_answer, factlabel = get_compare_val_answer(llm_answer, row["answer"])
        rgb_response_data_without_doc.at[index,'answer_without_doc'] = llm_answer
        result = 0 if 0 in labels else 1 if 1 in labels else None
        rgb_response_data_without_doc.at[index, 'label_without_doc'] = result
        create_json("llm_response_contra_wihtout_doc_72.json",index,llm_answer,row["query"],labels,row["answer"],factlabel)

        # Implement the tt logic
        if -1 in labels:
            insufficient_info += 1
        if 0 in labels or 1 in labels:
            execution += 1
        if factlabel == 1:
            fact_error_exe += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [29]:
# Filter rows where label_without_doc == 1
filtered_df = rgb_response_data_without_doc[rgb_response_data_without_doc['label_without_doc'] == 1]

# Compute proportion of label_without_doc == 1
accuracy = len(filtered_df) / len(rgb_response_data_without_doc)
print("Proportion of label_without_doc = 1:", accuracy)


Proportion of label_without_doc = 1: 0.94


# Execution for LLM with Positive_Wrong Documents

In [18]:
rgb_response_data_without_doc.sample(5)

,id,query,answer,fakeanswer,positive_wrong,positive,negative,answer_without_doc,label_without_doc
0,0,Super Bowl 2021 location,"Tampa, Florida","Glendale, Arizona","[The game was played on February 7, 2021, at R...","[The game was played on February 7, 2021, at R...",[Official Super Bowl LVIII Ticket Packages Now...,\n\nAnswer: Super Bowl LV was played at Raymon...,1
1,1,Which country won the most medals at the 2018 ...,Norway,U.S.,[U.S. set the record for most total medals at ...,[Norway set the record for most total medals a...,[PyeongChang's vision for the 2018 Games was t...,\n\nAnswer: **Norway** won the most medals at ...,1
2,2,Who acquired Instagram?,Facebook,Apple,"[Jul 27, 2022 ... When Apple made a bid to buy...","[Jul 27, 2022 ... When Facebook made a bid to ...",[The company remained independent up until it ...,\n\nAnswer: Instagram was acquired by **Facebo...,1
3,3,Who acquired WhatsApp?,Facebook,Apple,[Apple purchased WhatsApp in 2014. · Apple's b...,[Facebook purchased WhatsApp in 2014. · Facebo...,"[Jun 6, 2023 ... WhatsApp was founded in 2009 ...",\n\nAnswer: WhatsApp was acquired by **Faceboo...,1
4,4,Who won the women's singles Wimbledon in 2019?,Simona Halep,Angelique Kerber,[Angelique Kerber defeated Serena Williams in ...,[Simona Halep defeated Serena Williams in the ...,[Carlos Alcaraz wins the clinching point in th...,\n\nAnswer: Simona Halep \n\n,1
...,...,...,...,...,...,...,...,...,...
95,95,Who is the CEO of Apple in 2021?,Tim Cook,Steve Jobs,[Steve Jobs is Apple's CEO and a member of App...,[Tim Cook is Apple's CEO and a member of Apple...,[Apple chief executive (2011–present). After J...,\n\nAnswer: Tim Cook was the CEO of Apple in 2...,1
96,96,Who won the Masters golf tournament in 2021?,Hideki Matsuyama,Shane Lowry,"[Apr 12, 2021 ... Shane Lowry made history for...","[Apr 12, 2021 ... Hideki Matsuyama made histor...",[Official home of The 2023 Masters at Augusta ...,\n\nAnswer: **Hideki Matsuyama** won the Maste...,1
97,97,Who won the British Open golf tournament in 2020?,Shane Lowry,Hideki Matsuyama,"[Jul 13, 2021 ... The 2020 British Open was ca...","[Jul 13, 2021 ... The 2020 British Open was ca...",[The championship was won by Collin Morikawa w...,\n\nAnswer: **Shane Lowry** won the British Op...,1
98,98,which city hosted the olympic games in 2008?,Beijing,London,"[Under the direction of Liu Qi, London was ele...","[Under the direction of Liu Qi, Beijing was el...","[Dec 14, 2021 ... The costs of hosting the Oly...",\n\nAnswer: Beijing \n,1


In [19]:
sys_content_with_doc = "You are an accurate and reliable AI assistant that can answer questions with the help of external documents. Please note that external documents may contain noisy or factually incorrect information. If the information in the document contains the correct answer, you will give an accurate answer. If the information in the document does not contain the answer, you will generate ’I can not answer the question because of the insufficient information in documents.‘. If there are inconsistencies with the facts in some of the documents, please generate the response 'There are factual errors in the provided documents.' and provide the correct answer."

In [20]:
def response_prompt():
    retrieval_prompt= sys_content_with_doc +"Document:\n{documents} \n\nQuestion:\n{question}"
    retrieval_prompt_template = PromptTemplate.from_template(retrieval_prompt)
    return retrieval_prompt_template

In [21]:
def get_hugging_face_llm_response(documents,question,eval_model,temp):

     # Set up the LLM endpoint
    llm_retrieval = HuggingFaceEndpoint(
        repo_id=eval_model,
        temperature=temp,
        huggingfacehub_api_token=api_key,
    )

    # Combine the retrieval template and the LLM
    llm_retrieval_chain = response_prompt() | llm_retrieval

    # Prepare input for the chain
    input_retrieval = {
        "documents": documents,
        "question": question
    }

    # Invoke the chain to generate a response
    try:
        llm_response = llm_retrieval_chain.invoke(input_retrieval)
        return llm_response
    except Exception as e:
        print(f"Error invoking LLM retrieval chain: {e}")
        return None


In [22]:
rgb_response_data_doc['label_with_doc'] = None
rgb_response_data_doc['fact_level'] = None
rgb_response_data_doc['answer_with_doc'] = None

In [24]:
execution_1 = 0
insufficient_info_1 = 0 # Initialize the  counter
fact_error_exe_1=0

results = []  # List to store results for calculating the ratio later
response_model="Qwen/QwQ-32B-Preview"  #llama-3.3-70b-versatile,mixtral-8x7b-32768,Qwen/QwQ-32B-Preview,mistralai/Mistral-7B-Instruct-v0.3
temp = 0.7
for index, row in rgb_response_data_without_doc.iterrows():
    # Generate samples based on positive, negative data
    print(index)

    # docs = processdata(row)

    llm_answer = get_hugging_face_llm_response_new(row['positive_wrong'] ,row["query"], response_model,temp,api_key)

    if llm_answer is not None:
        # Compare the LLM answer with the ground truth
        labels, llm_answer, factlabel = get_compare_val_answer(llm_answer, row["answer"])
        rgb_response_data_without_doc.at[index, 'answer_with_doc'] = llm_answer
        rgb_response_data_without_doc.at[index, 'label_with_doc'] = labels
        rgb_response_data_without_doc.at[index, 'fact_level'] = factlabel

        create_json("llm_response_withdoc_32.json",index,llm_answer,row["query"],labels,row["answer"],factlabel)

        # Implement the tt logic
        if -1 in labels:
            insufficient_info_1 += 1
        if 0 in labels or 1 in labels:
            execution_1 += 1
        if factlabel == 1:
            fact_error_exe_1 += 1

# # print("Acccuracy with Doc") :  #rgb_response_data_without_doc['label_with_doc'].value_counts() = 1 the count of 1
# print(f"ED with Doc is {fact_error_exe_1}")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [32]:
print(f"ED with Doc is {fact_error_exe_1}")

ED with Doc is 0


In [33]:
rgb_response_data_without_doc['label_with_doc'].value_counts()

,count
label_with_doc,
0.0,77
1.0,23


In [34]:
rgb_response_data_without_doc['fact_level'].value_counts()

,count
fact_level,
0.0,100


In [35]:
# Filter where fact_level == 1 and other_column == 1
filtered_df = rgb_response_data_without_doc[(rgb_response_data_without_doc["fact_level"] == 1) & (rgb_response_data_without_doc["label_with_doc"] == 1)]
print("Correction rate :",filtered_df.shape[0]/fact_error_exe_1)

ZeroDivisionError: division by zero

In [ ]:
filtered_df.shape

(9, 12)